### **1.- Pre-procesamiento de Datos**
	1. Crea una tabla de datos que consolide las diferentes fuentes de datos del e-commerce para realizar el análisis correspondiente.
	Desarrolla un programa en Python (“1_1_oilst_processed.py”) para procesar las diferentes fuentes de datos del e-commerce en una tabla consolidada, la cual debe considerar la siguiente información en sus campos:
	Campos de las fuentes provistas:
	* order_id,customer_id
	* order_status
	* order_purchase_timestamp
	* order_approved_at
	* order_delivered_carrier_date
	* order_delivered_customer_date
	* order_estimated_delivery_date
	* distance_distribution_center
	* customer_unique_id
	* customer_zip_code_prefix
	* customer_city
	* customer_state
	* geolocation_zip_code_prefix
	* geolocation_lat
	* geolocation_lng
	* geolocation_city
	* geolocation_state
	* abbreviation
	* state_name

	**Campos calculados:**
	* total_products (total de productos de la orden)
	* total_sales (total monetario de la orden)
	* year (año en que se realiza el pedido)
	* month (mes en que se realiza el pedido)
	* quarter (trimestre en que se realiza el pedido)
	* year_month (año y mes en que se realiza el pedido)
	* delta_days (días transcurridos entre la fecha estimada de entrega y la entrega efectiva del pedido)
	* delay_status (indica si no hubo retraso, si se trató de un retraso corto menor o igual a tres días o uno largo, mayor a 3 días).

	Verifica que el script que has generado guarde la tabla de datos consolidada de las diferentes fuentes de datos del e-commerce en un archivo en formato .csv (“oilst_processed.csv”).

In [ ]:
# importamos librerías para la EDA
import os
import numpy as np
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Recursos
DATA_PATH= "/content/"
FILE_CUSTOMERS= 'olist_customers_dataset.xlsx'
FILE_ORDERS= 'olist_orders_dataset.csv'
FILE_GEOLOCATIONS= 'olist_geolocation_dataset.csv'
FILE_ORDER_ITEMS= 'olist_order_items_dataset.csv'
FILE_ORDER_PAYMENTS= 'olist_order_payments_dataset.csv'
FILE_STATES_ABREVIATION= 'states_abbreviations.json'
FILE_REGIONS= 'brasil_regions.csv'
FILE_GEODATA= 'brasil_geodata.json'

# 2.- Iniciamos la exploración de data de cada uno de los archivos

In [ ]:
# Probamos la existencia de los archivos imprimiendo sus rutas.
print(f"Ruta del archivo: {FILE_GEOLOCATIONS}")
print(os.path.join(DATA_PATH, FILE_GEOLOCATIONS))

Ruta del archivo: olist_geolocation_dataset.csv
/content/olist_geolocation_dataset.csv


# 2.1 Exploramos el archivo olist_customers_dataset.xlsx

In [ ]:
# Creamos un df de cada uno de los archivos
df_customers = pd.read_excel(os.path.join(DATA_PATH, FILE_CUSTOMERS))

In [ ]:
df_customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 5 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   customer_id               99441 non-null  object
 1   customer_unique_id        99441 non-null  object
 2   customer_zip_code_prefix  99441 non-null  int64 
 3   customer_city             99441 non-null  object
 4   customer_state            99441 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.8+ MB


In [ ]:
# cambiar el tipo de dato para customer_zip_code_prefix ya que de acuerdo al
# Anexo A el un tipo de dato numérico (texto)
df_customers['customer_zip_code_prefix'] = df_customers['customer_zip_code_prefix'].astype(str)
df_customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 5 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   customer_id               99441 non-null  object
 1   customer_unique_id        99441 non-null  object
 2   customer_zip_code_prefix  99441 non-null  object
 3   customer_city             99441 non-null  object
 4   customer_state            99441 non-null  object
dtypes: object(5)
memory usage: 3.8+ MB


In [ ]:
# Revisamos una muestra del df customers
df_customers.sample(10)

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
51061,f989b6b1bb5f0245a0280ff76a677f11,d3c3f16d67a1e9c58841d83a420da18b,30411,belo horizonte,MG
22249,b65714d110c92422f67d49614c32f883,230b3fa73170f4c20af25503af1b2873,3646,sao paulo,SP
81312,4536ccaeb1bc6078a3063b63c9e2905e,13a6bc772740dca2a4f38a5b5b6e3fb0,3922,sao paulo,SP
49538,19970def56bdc13635f5591fc47a3727,2da9c684118fafce96940108437b2480,37720,botelhos,MG
87820,77a15f7a0bb69e8b0aecf36b112f9be0,3d4cd13fd525234c1c4a2304fbe769e0,2452,sao paulo,SP
5863,66df73c74be57471c69e6e20c9490a90,f55be2f0324b5b12eee35c1a9379b20e,95727,boa vista do sul,RS
52992,308532b0d57d2a3cf82dd24a7bbc8ee8,b6d7faaf236059897f4d0440e1a327c7,18206,itapetininga,SP
81185,c3f03ad00c3a59d7e1be8376fcd6e9f2,dcfc9c44bf4471c67137fac8b54bce59,22221,rio de janeiro,RJ
74621,36efd32104e8b48038e2f4dee07c3935,813eee947cafda0ed444814b805325f1,31260,belo horizonte,MG
21166,fa37c71b257f8a67ce2e7018b3aeac32,a2bdda78bde40cf6a6f02a0171885daa,37160,campos gerais,MG


In [ ]:
# Revisamos las primeras 5 filas del df customers
df_customers.head(5)

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP


In [ ]:
# Revisamos las últimas 5 filas del df customers
df_customers.tail()

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
99436,17ddf5dd5d51696bb3d7c6291687be6f,1a29b476fee25c95fbafc67c5ac95cf8,3937,sao paulo,SP
99437,e7b71a9017aa05c9a7fd292d714858e8,d52a67c98be1cf6a5c84435bd38d095d,6764,taboao da serra,SP
99438,5e28dfe12db7fb50a4b2f691faecea5e,e9f50caf99f032f0bf3c55141f019d99,60115,fortaleza,CE
99439,56b18e2166679b8a959d72dd06da27f9,73c2643a0a458b49f58cea58833b192e,92120,canoas,RS
99440,274fa6071e5e17fe303b9748641082c8,84732c5050c01db9b23e19ba39899398,6703,cotia,SP


In [ ]:
# valores únicos para cada una de las columnas
df_customers.nunique()

,0
customer_id,99441
customer_unique_id,96096
customer_zip_code_prefix,14994
customer_city,4119
customer_state,27


In [ ]:
df_customers.describe()

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
count,99441,99441,99441,99441,99441
unique,99441,96096,14994,4119,27
top,06b8999e2fba1a1fbc88172c00ba8bc7,8d50f5eadf50201ccdcedfb9e2ac8455,22790,sao paulo,SP
freq,1,17,142,15540,41746


Con esta función detectamos que el customer_unique_id unique son 96096 registros y que el top es el 8d50f5eadf50201ccdcedfb9e2ac8455, por lo que sería interesante explorar sus registros ordenados por customer_id

In [ ]:
df_customers[df_customers['customer_unique_id'] == '8d50f5eadf50201ccdcedfb9e2ac8455'].sort_values(by='customer_id')

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
90268,0bf8bf19944a7f8b40ba86fef778ca7c,8d50f5eadf50201ccdcedfb9e2ac8455,4045,sao paulo,SP
74510,0e4fdc084a6b9329ed55d62dcd653ccf,8d50f5eadf50201ccdcedfb9e2ac8455,4045,sao paulo,SP
14186,1bd3585471932167ab72a84955ebefea,8d50f5eadf50201ccdcedfb9e2ac8455,4045,sao paulo,SP
52574,1c62b48fb34ee043310dcb233caabd2e,8d50f5eadf50201ccdcedfb9e2ac8455,4045,sao paulo,SP
96652,31dd055624c66f291578297a551a6cdf,8d50f5eadf50201ccdcedfb9e2ac8455,4045,sao paulo,SP
72745,3414a9c813e3ca02504b8be8b2deb27f,8d50f5eadf50201ccdcedfb9e2ac8455,4045,sao paulo,SP
38073,42dbc1ad9d560637c9c4c1533746f86d,8d50f5eadf50201ccdcedfb9e2ac8455,4045,sao paulo,SP
67996,6289b75219d757a56c0cce8d9e427900,8d50f5eadf50201ccdcedfb9e2ac8455,4045,sao paulo,SP
48614,65f9db9dd07a4e79b625effa4c868fcb,8d50f5eadf50201ccdcedfb9e2ac8455,4045,sao paulo,SP
16654,897b7f72042714efaa64ac306ba0cafc,8d50f5eadf50201ccdcedfb9e2ac8455,4045,sao paulo,SP


# 2.2 Exploramos el archivo olist_orders_dataset.xlsx

In [ ]:
#creamos el df de lista de ordenes
df_olist_orders = pd.read_csv(os.path.join(DATA_PATH, FILE_ORDERS))
df_olist_orders

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,distance_distribution_center
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,29.84
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00,25.53
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00,61.56
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00,60.30
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00,80.97
...,...,...,...,...,...,...,...,...,...
99436,9c5dedf39a927c1b2549525ed64a053c,39bd1228ee8140590ac3aca26f2dfe00,delivered,2017-03-09 09:54:05,2017-03-09 09:54:05,2017-03-10 11:18:03,2017-03-17 15:08:01,2017-03-28 00:00:00,68.26
99437,63943bddc261676b46f01ca7ac2f7bd8,1fca14ff2861355f6e5f14306ff977a7,delivered,2018-02-06 12:58:58,2018-02-06 13:10:37,2018-02-07 23:22:42,2018-02-28 17:37:56,2018-03-02 00:00:00,14.92
99438,83c1379a015df1e13d02aae0204711ab,1aa71eb042121263aafbe80c1b562c9c,delivered,2017-08-27 14:46:43,2017-08-27 15:04:16,2017-08-28 20:52:26,2017-09-21 11:24:17,2017-09-27 00:00:00,36.37
99439,11c177c8e97725db2631073c19f07b62,b331b74b18dc79bcdf6532d51e1637c1,delivered,2018-01-08 21:28:27,2018-01-08 21:36:21,2018-01-12 15:35:03,2018-01-25 23:32:54,2018-02-15 00:00:00,32.04


In [ ]:
df_olist_orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 9 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   order_id                       99441 non-null  object 
 1   customer_id                    99441 non-null  object 
 2   order_status                   99441 non-null  object 
 3   order_purchase_timestamp       99441 non-null  object 
 4   order_approved_at              99281 non-null  object 
 5   order_delivered_carrier_date   97658 non-null  object 
 6   order_delivered_customer_date  96476 non-null  object 
 7   order_estimated_delivery_date  99441 non-null  object 
 8   distance_distribution_center   96470 non-null  float64
dtypes: float64(1), object(8)
memory usage: 6.8+ MB


De acuerdo al Anexo A los campos de fecha deben ser de tipo timestamp por lo que debemos convertirlos al formato esperado.

In [ ]:
# Creamos una lista de columnas de tipo timestamp
date_columns = ['order_purchase_timestamp',
                'order_approved_at',
                'order_delivered_carrier_date',
                'order_delivered_customer_date',
                'order_estimated_delivery_date'
                ]
for col in date_columns:
  df_olist_orders[col] = pd.to_datetime(df_olist_orders[col],
                                        infer_datetime_format=True
                                        )



In [ ]:
  # Validamos que tengamos el nuevo formato
  df_olist_orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 9 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   order_id                       99441 non-null  object        
 1   customer_id                    99441 non-null  object        
 2   order_status                   99441 non-null  object        
 3   order_purchase_timestamp       99441 non-null  datetime64[ns]
 4   order_approved_at              99281 non-null  datetime64[ns]
 5   order_delivered_carrier_date   97658 non-null  datetime64[ns]
 6   order_delivered_customer_date  96476 non-null  datetime64[ns]
 7   order_estimated_delivery_date  99441 non-null  datetime64[ns]
 8   distance_distribution_center   96470 non-null  float64       
dtypes: datetime64[ns](5), float64(1), object(3)
memory usage: 6.8+ MB


Como

In [ ]:
df_olist_orders.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,distance_distribution_center
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18,29.84
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13,25.53
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04,61.56
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15,60.30
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26,80.97


In [ ]:
df_olist_orders.tail()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,distance_distribution_center
99436,9c5dedf39a927c1b2549525ed64a053c,39bd1228ee8140590ac3aca26f2dfe00,delivered,2017-03-09 09:54:05,2017-03-09 09:54:05,2017-03-10 11:18:03,2017-03-17 15:08:01,2017-03-28,68.26
99437,63943bddc261676b46f01ca7ac2f7bd8,1fca14ff2861355f6e5f14306ff977a7,delivered,2018-02-06 12:58:58,2018-02-06 13:10:37,2018-02-07 23:22:42,2018-02-28 17:37:56,2018-03-02,14.92
99438,83c1379a015df1e13d02aae0204711ab,1aa71eb042121263aafbe80c1b562c9c,delivered,2017-08-27 14:46:43,2017-08-27 15:04:16,2017-08-28 20:52:26,2017-09-21 11:24:17,2017-09-27,36.37
99439,11c177c8e97725db2631073c19f07b62,b331b74b18dc79bcdf6532d51e1637c1,delivered,2018-01-08 21:28:27,2018-01-08 21:36:21,2018-01-12 15:35:03,2018-01-25 23:32:54,2018-02-15,32.04
99440,66dea50a8b16d9b4dee7af250b4be1a5,edb027a75a1449115f6b43211ae02a24,delivered,2018-03-08 20:57:30,2018-03-09 11:20:28,2018-03-09 22:11:59,2018-03-16 13:08:30,2018-04-03,3.54


En este df se encuentra la información relevante para crear columnas calculadas solicitadas en el requerimiento:

1.   year (año en que se realiza el pedido)
2.   month (mes en que se realiza el pedido)
3.   quarter (trimestre en que se realiza el pedido)
4.   year_month (año y mes en que se realiza el pedido)
5.   delta_days (días transcurridos entre la fecha estimada de entrega y la   entrega efectiva del pedido)
6.   delay_status (indica si no hubo retraso, si se trató de un retraso corto menor o igual a tres días o uno largo, mayor a 3 días)

In [ ]:
#Iniciamos con las que solo requiere hacer un cast
df_olist_orders['year'] = df_olist_orders['order_purchase_timestamp'].dt.year
df_olist_orders['month'] = df_olist_orders['order_purchase_timestamp'].dt.month
df_olist_orders['quarter'] = df_olist_orders['order_purchase_timestamp'].dt.to_period('Q')
df_olist_orders['year_month'] = df_olist_orders['order_purchase_timestamp'].dt.to_period('M')

In [ ]:
df_olist_orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 13 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   order_id                       99441 non-null  object        
 1   customer_id                    99441 non-null  object        
 2   order_status                   99441 non-null  object        
 3   order_purchase_timestamp       99441 non-null  datetime64[ns]
 4   order_approved_at              99281 non-null  datetime64[ns]
 5   order_delivered_carrier_date   97658 non-null  datetime64[ns]
 6   order_delivered_customer_date  96476 non-null  datetime64[ns]
 7   order_estimated_delivery_date  99441 non-null  datetime64[ns]
 8   distance_distribution_center   96470 non-null  float64       
 9   year                           99441 non-null  int32         
 10  month                          99441 non-null  int32         
 11  quarter        

Podemos observar que se crearon las nuevas columnas que son de tipo int32 y period.

In [ ]:
df_olist_orders[['order_id','year', 'month', 'quarter', 'year_month']]

,order_id,year,month,quarter,year_month
0,e481f51cbdc54678b7cc49136f2d6af7,2017,10,2017Q4,2017-10
1,53cdb2fc8bc7dce0b6741e2150273451,2018,7,2018Q3,2018-07
2,47770eb9100c2d0c44946d9cf07ec65d,2018,8,2018Q3,2018-08
3,949d5b44dbf5de918fe9c16f97b45f8a,2017,11,2017Q4,2017-11
4,ad21c59c0840e6cb83a9ceb5573f8159,2018,2,2018Q1,2018-02
...,...,...,...,...,...
99436,9c5dedf39a927c1b2549525ed64a053c,2017,3,2017Q1,2017-03
99437,63943bddc261676b46f01ca7ac2f7bd8,2018,2,2018Q1,2018-02
99438,83c1379a015df1e13d02aae0204711ab,2017,8,2017Q3,2017-08
99439,11c177c8e97725db2631073c19f07b62,2018,1,2018Q1,2018-01


In [ ]:
df_olist_orders.describe()

,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,distance_distribution_center,year,month
count,99441,99281,97658,96476,99441,9.647000e+04,99441.000000,99441.000000
mean,2017-12-31 08:43:12.776581120,2017-12-31 18:35:24.098800128,2018-01-04 21:49:48.138278656,2018-01-14 12:09:19.035542272,2018-01-24 03:08:37.730111232,3.899211e+02,2017.539838,6.032220
min,2016-09-04 21:15:19,2016-09-15 12:16:38,2016-10-08 10:34:01,2016-10-11 13:46:32,2016-09-30 00:00:00,1.000000e-02,2016.000000,1.000000
25%,2017-09-12 14:46:19,2017-09-12 23:24:16,2017-09-15 22:28:50.249999872,2017-09-25 22:07:22.249999872,2017-10-03 00:00:00,2.351250e+01,2017.000000,3.000000
50%,2018-01-18 23:04:36,2018-01-19 11:36:13,2018-01-24 16:10:58,2018-02-02 19:28:10.500000,2018-02-15 00:00:00,5.039000e+01,2018.000000,6.000000
75%,2018-05-04 15:42:16,2018-05-04 20:35:10,2018-05-08 13:37:45,2018-05-15 22:48:52.249999872,2018-05-25 00:00:00,7.713000e+01,2018.000000,8.000000
max,2018-10-17 17:30:18,2018-09-03 17:40:06,2018-09-11 19:48:28,2018-10-17 13:22:46,2018-11-12 00:00:00,1.349722e+06,2018.000000,12.000000
std,NaN,NaN,NaN,NaN,NaN,1.312301e+04,0.505007,3.232999


Crear la columna calculada delta_days que se conforma de la diferencia enter la columna order_delivered_customer_date y order_estimated_delivery_date

In [ ]:
#calculamos en días
df_olist_orders['delta_days'] = (df_olist_orders['order_delivered_customer_date'] - df_olist_orders['order_estimated_delivery_date']).dt.total_seconds()/ 60 / 60 / 24
df_olist_orders[['order_delivered_customer_date','order_estimated_delivery_date','delta_days']]

,order_delivered_customer_date,order_estimated_delivery_date,delta_days
0,2017-10-10 21:25:13,2017-10-18,-7.107488
1,2018-08-07 15:27:45,2018-08-13,-5.355729
2,2018-08-17 18:06:29,2018-09-04,-17.245498
3,2017-12-02 00:28:42,2017-12-15,-12.980069
4,2018-02-16 18:17:02,2018-02-26,-9.238171
...,...,...,...
99436,2017-03-17 15:08:01,2017-03-28,-10.369433
99437,2018-02-28 17:37:56,2018-03-02,-1.265324
99438,2017-09-21 11:24:17,2017-09-27,-5.524803
99439,2018-01-25 23:32:54,2018-02-15,-20.018819


In [ ]:
#Exploramos las estadísticas de la nueva columna
df_olist_orders['delta_days'].describe()

,delta_days
count,96476.000000
mean,-11.179120
std,10.186113
min,-146.016123
25%,-16.244384
50%,-11.948941
75%,-6.390000
max,188.975081


Observaciones:

Los valores negativos significa que el pedido llegó antes de lo estimado.

Podemos ver que el pedido que llegó con mayor anticipación fue de -146 días y el pedido que tardó más en llegar fue de más de 188 días.


In [ ]:
#Analizamos el top 5 de los pedidos en el que se entregaron antes de lo estimado
df_olist_orders[df_olist_orders['delta_days'] < 0].sort_values(by='delta_days',  ascending=True).head(5)

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,distance_distribution_center,year,month,quarter,year_month,delta_days
40094,0607f0efea4b566f1eb8f7d3c2397320,a5fbb6579eacbeb02752a143bfe82405,delivered,2018-03-06 09:47:07,2018-03-06 09:55:47,2018-03-07 21:33:39,2018-03-09 23:36:47,2018-08-03,0.22,2018,3,2018Q1,2018-03,-146.016123
15791,c72727d29cde4cf870d569bf65edabfd,964253ff0e4e08180064764a450e521f,delivered,2017-02-07 18:01:15,2017-02-09 02:50:07,2017-02-10 05:32:55,2017-02-14 14:27:45,2017-07-04,13.25,2017,2,2017Q1,2017-02,-139.397396
57160,eec7f369423b033e549c02f3c5381205,32cef4bdd6bfa50612d81dc77a93b131,delivered,2018-02-06 20:44:56,2018-02-08 10:10:31,2018-02-15 20:42:12,2018-02-27 16:35:43,2018-07-12,54.56,2018,2,2018Q1,2018-02,-134.308530
86444,c2bb89b5c1dd978d507284be78a04cb2,6357fffb5704244d552615bbfcea1442,delivered,2017-05-23 22:28:36,2017-05-24 22:35:08,2017-05-29 02:03:28,2017-06-09 13:35:54,2017-10-11,98.33,2017,5,2017Q2,2017-05,-123.433403
67488,40dc2ba6f322a17626aac6244332828c,6210a37f9d6a265a4f3fbe2c21672a00,delivered,2017-10-05 21:39:05,2017-10-05 21:49:21,2017-10-09 12:13:58,2017-10-13 13:49:07,2018-01-30,43.47,2017,10,2017Q4,2017-10,-108.424225


In [ ]:
#Analizamos el top 5 de los pedidos en el que se entregaron más tarde de lo estimado
df_olist_orders[df_olist_orders['delta_days'] > 0].sort_values(by='delta_days',  ascending=False).head(5)

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,distance_distribution_center,year,month,quarter,year_month,delta_days
55619,1b3190b2dfa9d789e1f14c05b647a14a,d306426abe5fca15e54b645e4462dc7b,delivered,2018-02-23 14:57:35,2018-02-23 15:16:14,2018-02-26 18:49:07,2018-09-19 23:24:07,2018-03-15,1349721.75,2018,2,2018Q1,2018-02,188.975081
19590,ca07593549f1816d26a572e06dc1eab6,75683a92331068e2d281b11a7866ba44,delivered,2017-02-21 23:31:27,2017-02-23 02:35:15,2017-03-08 13:47:46,2017-09-19 14:36:39,2017-03-22,1197957.02,2017,2,2017Q1,2017-02,181.608785
11399,47b40429ed8cce3aee9199792275433f,cb2caaaead400c97350c37a3fc536867,delivered,2018-01-03 09:44:01,2018-01-03 10:31:15,2018-02-06 01:48:28,2018-07-13 20:51:31,2018-01-19,1087925.34,2018,1,2018Q1,2018-01,175.869109
81401,2fe324febf907e3ea3f2aa9650869fa5,65b14237885b3972ebec28c0f7dd2220,delivered,2017-03-13 20:17:10,2017-03-13 20:17:10,2017-03-17 07:23:53,2017-09-19 17:00:07,2017-04-05,943398.70,2017,3,2017Q1,2017-03,167.708414
89130,285ab9426d6982034523a855f55a885e,9cf2c3fa2632cee748e1a59ca9d09b21,delivered,2017-03-08 22:47:40,2017-03-08 22:47:40,2017-03-09 08:53:20,2017-09-19 14:00:04,2017-04-06,924540.23,2017,3,2017Q1,2017-03,166.583380


Lo siguiente es definir la columna delay_status donde un valor negativo en delta_days mayor a 0 días pero menor a 3 quiere decir que tiene un atraso de entrega dentro de lo aceptable, pero si delta_days es mayor a 3, significa que se tiene un atraso significativo.

In [ ]:
df_olist_orders['delay_status'] = np.where(
    df_olist_orders['delta_days'] > 3, 'long_delay',
    np.where(
        df_olist_orders['delta_days'] <= 0, 'on_time',
        'short_delay'
    )
)

In [ ]:
df_olist_orders[['order_id','delta_days','delay_status']]

,order_id,delta_days,delay_status
0,e481f51cbdc54678b7cc49136f2d6af7,-7.107488,on_time
1,53cdb2fc8bc7dce0b6741e2150273451,-5.355729,on_time
2,47770eb9100c2d0c44946d9cf07ec65d,-17.245498,on_time
3,949d5b44dbf5de918fe9c16f97b45f8a,-12.980069,on_time
4,ad21c59c0840e6cb83a9ceb5573f8159,-9.238171,on_time
...,...,...,...
99436,9c5dedf39a927c1b2549525ed64a053c,-10.369433,on_time
99437,63943bddc261676b46f01ca7ac2f7bd8,-1.265324,on_time
99438,83c1379a015df1e13d02aae0204711ab,-5.524803,on_time
99439,11c177c8e97725db2631073c19f07b62,-20.018819,on_time


In [ ]:
#Exploramos datos estadísticos de la nueva columna
df_olist_orders['delay_status'].describe()

,delay_status
count,99441
unique,3
top,on_time
freq,88649


In [ ]:
#Valores distintos
df_olist_orders['delay_status'].value_counts()

,count
delay_status,
on_time,88649
short_delay,5627
long_delay,5165


# 2.3 Exploramos el archivo olist_geolocation_dataset.csv

In [ ]:
#convertimos directamente la columna geolocation_zip_cod_prefix en tipo str
df_geolocations = pd.read_csv(os.path.join(DATA_PATH, FILE_GEOLOCATIONS), dtype={'geolocation_zip_code_prefix': 'str'})
df_geolocations

,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
0,01037,-23.545621,-46.639292,sao paulo,SP
1,01046,-23.546081,-46.644820,sao paulo,SP
2,01046,-23.546129,-46.642951,sao paulo,SP
3,01041,-23.544392,-46.639499,sao paulo,SP
4,01035,-23.541578,-46.641607,sao paulo,SP
...,...,...,...,...,...
1000158,99950,-28.068639,-52.010705,tapejara,RS
1000159,99900,-27.877125,-52.224882,getulio vargas,RS
1000160,99950,-28.071855,-52.014716,tapejara,RS
1000161,99980,-28.388932,-51.846871,david canabarro,RS


In [ ]:
#Obtenemos información del df
df_geolocations.info()
#notamos que todos coinciden en la cantidad de registros no nulos.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000163 entries, 0 to 1000162
Data columns (total 5 columns):
 #   Column                       Non-Null Count    Dtype  
---  ------                       --------------    -----  
 0   geolocation_zip_code_prefix  1000163 non-null  object 
 1   geolocation_lat              1000163 non-null  float64
 2   geolocation_lng              1000163 non-null  float64
 3   geolocation_city             1000163 non-null  object 
 4   geolocation_state            1000163 non-null  object 
dtypes: float64(2), object(3)
memory usage: 38.2+ MB


In [ ]:
df_geolocations.describe()

,geolocation_lat,geolocation_lng
count,1.000163e+06,1.000163e+06
mean,-2.117615e+01,-4.639054e+01
std,5.715866e+00,4.269748e+00
min,-3.660537e+01,-1.014668e+02
25%,-2.360355e+01,-4.857317e+01
50%,-2.291938e+01,-4.663788e+01
75%,-1.997962e+01,-4.376771e+01
max,4.506593e+01,1.211054e+02


In [ ]:
#dado que es un df de codigos postales revisamos una estadística la columna de codigo postal
df_geolocations['geolocation_zip_code_prefix'].describe()

,geolocation_zip_code_prefix
count,1000163
unique,19015
top,24220
freq,1146


Podemos notar que el CP 24220 está en el top con una frecuencia de 1146 ocasiones, por lo que podemos revisar los registros de este CP para determinar si se puede considerar la eliminación de CP duplicados.

In [ ]:
df_geolocations.query('geolocation_zip_code_prefix == "24220"')

,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
470805,24220,-22.905817,-43.106989,niteroi,RJ
470811,24220,-22.902306,-43.112545,niteroi,RJ
470812,24220,-22.904567,-43.110491,niteroi,RJ
470820,24220,-22.902575,-43.109192,niteroi,RJ
470821,24220,-22.907500,-43.106170,niteroi,RJ
...,...,...,...,...,...
474265,24220,-22.904023,-43.111683,niteroi,RJ
474266,24220,-22.905393,-43.100512,niterói,RJ
474269,24220,-22.906420,-43.104933,niteroi,RJ
474270,24220,-22.909701,-43.108452,niteroi,RJ


Dado que el CP para nuestro análisis debe ser único podemos procedemos a crear un nuevo df en la que vamos a eliminar los registros duplicados del df basados por la columna geolocation_zip_code_prefix

In [ ]:
df_unique_geolocations = df_geolocations.drop_duplicates(subset=['geolocation_zip_code_prefix'])
df_unique_geolocations

,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
0,01037,-23.545621,-46.639292,sao paulo,SP
1,01046,-23.546081,-46.644820,sao paulo,SP
3,01041,-23.544392,-46.639499,sao paulo,SP
4,01035,-23.541578,-46.641607,sao paulo,SP
5,01012,-23.547762,-46.635361,são paulo,SP
...,...,...,...,...,...
999774,99955,-28.107588,-52.144019,vila langaro,RS
999780,99970,-28.345143,-51.876926,ciriaco,RS
999786,99910,-27.863500,-52.084760,floriano peixoto,RS
999803,99920,-27.858716,-52.300403,erebango,RS


In [ ]:
#El nuevo df ya tiene estrictamente valores unicos en la columna de CP
df_unique_geolocations.info()

<class 'pandas.core.frame.DataFrame'>
Index: 19015 entries, 0 to 999846
Data columns (total 5 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   geolocation_zip_code_prefix  19015 non-null  object 
 1   geolocation_lat              19015 non-null  float64
 2   geolocation_lng              19015 non-null  float64
 3   geolocation_city             19015 non-null  object 
 4   geolocation_state            19015 non-null  object 
dtypes: float64(2), object(3)
memory usage: 891.3+ KB


In [ ]:
df_unique_geolocations['geolocation_zip_code_prefix'].describe()

,geolocation_zip_code_prefix
count,19015
unique,19015
top,01037
freq,1


# 2.4 Exploramos el archivo olist_order_items_dataset.csv

In [ ]:
#Comprobamos la existencia del archivo
print(f"Ruta del archivo: {FILE_ORDER_ITEMS}")
print(os.path.join(DATA_PATH, FILE_ORDER_ITEMS))
os.path.exists(os.path.join(DATA_PATH, FILE_ORDER_ITEMS))

Ruta del archivo: olist_order_items_dataset.csv
/content/olist_order_items_dataset.csv


True

In [ ]:
#Creamos el df
df_order_items = pd.read_csv(os.path.join(DATA_PATH, FILE_ORDER_ITEMS))
df_order_items

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14
...,...,...,...,...,...,...,...
112645,fffc94f6ce00a00581880bf54a75a037,1,4aa6014eceb682077f9dc4bffebc05b0,b8bc237ba3788b23da09c0f1f3a3288c,2018-05-02 04:11:01,299.99,43.41
112646,fffcd46ef2263f404302a634eb57f7eb,1,32e07fd915822b0765e448c4dd74c828,f3c38ab652836d21de61fb8314b69182,2018-07-20 04:31:48,350.00,36.53
112647,fffce4705a9662cd70adb13d4a31832d,1,72a30483855e2eafc67aee5dc2560482,c3cfdc648177fdbbbb35635a37472c53,2017-10-30 17:14:25,99.90,16.95
112648,fffe18544ffabc95dfada21779c9644f,1,9c422a519119dcad7575db5af1ba540e,2b3e4a2a3ea8e01938cabda2a3e5cc79,2017-08-21 00:04:32,55.99,8.72


In [ ]:
#Obtenemos la información relevante
df_order_items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112650 entries, 0 to 112649
Data columns (total 7 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   order_id             112650 non-null  object 
 1   order_item_id        112650 non-null  int64  
 2   product_id           112650 non-null  object 
 3   seller_id            112650 non-null  object 
 4   shipping_limit_date  112650 non-null  object 
 5   price                112650 non-null  float64
 6   freight_value        112650 non-null  float64
dtypes: float64(2), int64(1), object(4)
memory usage: 6.0+ MB


In [ ]:
#Estadísticas descriptivas de las columnas numéricas
df_order_items.describe()

,order_item_id,price,freight_value
count,112650.000000,112650.000000,112650.000000
mean,1.197834,120.653739,19.990320
std,0.705124,183.633928,15.806405
min,1.000000,0.850000,0.000000
25%,1.000000,39.900000,13.080000
50%,1.000000,74.990000,16.260000
75%,1.000000,134.900000,21.150000
max,21.000000,6735.000000,409.680000


El requerimiento solicita cantidad de productos que hay en la orden, por lo que debemos realizar un agrupamiento por order_id, realizando un count de la columna order_item_id.
Adicional se solicita obtener el total del precio de la orden el cual es la sumatoria de la columna price

In [ ]:
order_items_agg = df_order_items.groupby('order_id').agg({'order_item_id': 'count', 'price': 'sum'}).reset_index()
order_items_agg

,order_id,order_item_id,price
0,00010242fe8c5a6d1ba2dd792cb16214,1,58.90
1,00018f77f2f0320c557190d7a144bdd3,1,239.90
2,000229ec398224ef6ca0657da4fc703e,1,199.00
3,00024acbcdf0a6daa1e931b038114c75,1,12.99
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,199.90
...,...,...,...
98661,fffc94f6ce00a00581880bf54a75a037,1,299.99
98662,fffcd46ef2263f404302a634eb57f7eb,1,350.00
98663,fffce4705a9662cd70adb13d4a31832d,1,99.90
98664,fffe18544ffabc95dfada21779c9644f,1,55.99


In [ ]:
order_items_agg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98666 entries, 0 to 98665
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   order_id       98666 non-null  object 
 1   order_item_id  98666 non-null  int64  
 2   price          98666 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 2.3+ MB


In [ ]:
#renombramos columnas para dejarlas de acuerdo al requerimiento
order_items_agg.rename(columns={'order_item_id': 'total_products', 'price': 'total_sales'}, inplace=True)
order_items_agg

,order_id,total_products,total_sales
0,00010242fe8c5a6d1ba2dd792cb16214,1,58.90
1,00018f77f2f0320c557190d7a144bdd3,1,239.90
2,000229ec398224ef6ca0657da4fc703e,1,199.00
3,00024acbcdf0a6daa1e931b038114c75,1,12.99
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,199.90
...,...,...,...
98661,fffc94f6ce00a00581880bf54a75a037,1,299.99
98662,fffcd46ef2263f404302a634eb57f7eb,1,350.00
98663,fffce4705a9662cd70adb13d4a31832d,1,99.90
98664,fffe18544ffabc95dfada21779c9644f,1,55.99


In [ ]:
#observamos de forma descendente las ordenes con mayor cantidad de productos
order_items_agg.sort_values(by='total_products', ascending=False)

,order_id,total_products,total_sales
50137,8272b63d03f5f79c56e9e4120aec44ef,21,31.80
10459,1b15974a0141d54e36626dca3fdc731a,20,2000.00
65715,ab14fdcfbe524636d65ee38360e22ce8,20,1974.00
60941,9ef13efd6949e4573a18964dd1bbe7f5,15,765.00
25583,428a2f660dc84138d969ccd69a0ab6d5,15,982.35
...,...,...,...
34640,5a0911d70c1f85d3bed0df1bf693a6dd,1,59.00
34639,5a082b558a3798d3e36d93bfa8ca1eae,1,9.00
34637,5a07264682e0b8fbb3f166edbbffc6e8,1,19.90
34636,5a071192a28951b76774e5a760c8c9b7,1,899.00


# 2.5 Exploramos el archivo olist_order_payments_dataset.csv

In [ ]:
#validamos la existencia del archivo
print(f"Ruta del archivo: {FILE_ORDER_PAYMENTS}")
print(os.path.join(DATA_PATH, FILE_ORDER_PAYMENTS))
os.path.exists(os.path.join(DATA_PATH, FILE_ORDER_PAYMENTS))

Ruta del archivo: olist_order_payments_dataset.csv
/content/olist_order_payments_dataset.csv


True

In [ ]:
#creamos el df
df_order_payments = pd.read_csv(os.path.join(DATA_PATH, FILE_ORDER_PAYMENTS))
df_order_payments

,order_id,payment_sequential,payment_type,payment_installments,payment_value
0,b81ef226f3fe1789b1e8b2acac839d17,1,credit_card,8,99.33
1,a9810da82917af2d9aefd1278f1dcfa0,1,credit_card,1,24.39
2,25e8ea4e93396b6fa0d3dd708e76c1bd,1,credit_card,1,65.71
3,ba78997921bbcdc1373bb41e913ab953,1,credit_card,8,107.78
4,42fdf880ba16b47b59251dd489d4441a,1,credit_card,2,128.45
...,...,...,...,...,...
103881,0406037ad97740d563a178ecc7a2075c,1,boleto,1,363.31
103882,7b905861d7c825891d6347454ea7863f,1,credit_card,2,96.80
103883,32609bbb3dd69b3c066a6860554a77bf,1,credit_card,1,47.77
103884,b8b61059626efa996a60be9bb9320e10,1,credit_card,5,369.54


In [ ]:
df_order_payments.info()  #observamos la información del df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103886 entries, 0 to 103885
Data columns (total 5 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   order_id              103886 non-null  object 
 1   payment_sequential    103886 non-null  int64  
 2   payment_type          103886 non-null  object 
 3   payment_installments  103886 non-null  int64  
 4   payment_value         103886 non-null  float64
dtypes: float64(1), int64(2), object(2)
memory usage: 4.0+ MB


In [ ]:
df_order_payments['payment_type'].value_counts()  #observamos los distinto tipos de pagos

,count
payment_type,
credit_card,76795
boleto,19784
voucher,5775
debit_card,1529
not_defined,3


# 2.6 Exploramos el archivo states_abreviation.json

In [ ]:
#validamos la existencia del archivo
print(f"Ruta del archivo: {FILE_STATES_ABREVIATION}")
print(os.path.join(DATA_PATH, FILE_STATES_ABREVIATION))
os.path.exists(os.path.join(DATA_PATH, FILE_STATES_ABREVIATION))

Ruta del archivo: states_abbreviations.json
/content/states_abbreviations.json


True

In [ ]:
#creamos el df
df_states_abreviation = pd.read_json(os.path.join(DATA_PATH, FILE_STATES_ABREVIATION))
df_states_abreviation.head()

,abbreviation,state_name
0,AC,Acre
1,AL,Alagoas
2,AP,Amapá
3,AM,Amazonas
4,BA,Bahia


In [ ]:
df_states_abreviation.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27 entries, 0 to 26
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   abbreviation  27 non-null     object
 1   state_name    27 non-null     object
dtypes: object(2)
memory usage: 560.0+ bytes


# 2.7 Exploramos el archivo brasil_regions.csv

In [ ]:
#validamos que exista el archivo
print(f"Ruta del archivo: {FILE_REGIONS}")
print(os.path.join(DATA_PATH, FILE_REGIONS))
os.path.exists(os.path.join(DATA_PATH, FILE_REGIONS))

Ruta del archivo: brasil_regions.csv
/content/brasil_regions.csv


True

In [ ]:
#creamos el df
df_regions = pd.read_csv(os.path.join(DATA_PATH, FILE_REGIONS))
df_regions.head()

,state_name,abbreviation,region
0,Rondônia,RO,north
1,Acre,AC,north
2,Amazonas,AM,north
3,Roraima,RR,north
4,Pará,PA,north


In [ ]:
df_regions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27 entries, 0 to 26
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   state_name    27 non-null     object
 1   abbreviation  27 non-null     object
 2   region        27 non-null     object
dtypes: object(3)
memory usage: 776.0+ bytes


# 2.8 Exploramos el archivo brasil_geodata.json

In [ ]:
#validamos que exista el archivo
print(f"Ruta del archivo: {FILE_GEODATA}")
print(os.path.join(DATA_PATH, FILE_GEODATA))
os.path.exists(os.path.join(DATA_PATH, FILE_GEODATA))

Ruta del archivo: brasil_geodata.json
/content/brasil_geodata.json


True

In [ ]:
#creamos el df
df_geodata = pd.read_json(os.path.join(DATA_PATH, FILE_GEODATA))
df_geodata.head()

,type,features
0,FeatureCollection,"{'type': 'Feature', 'id': 0, 'properties': {'U..."
1,FeatureCollection,"{'type': 'Feature', 'id': 1, 'properties': {'U..."
2,FeatureCollection,"{'type': 'Feature', 'id': 2, 'properties': {'U..."
3,FeatureCollection,"{'type': 'Feature', 'id': 3, 'properties': {'U..."
4,FeatureCollection,"{'type': 'Feature', 'id': 4, 'properties': {'U..."


# 3.- Unión de archivos para generar entregable

In [ ]:
#Creamos un dataframe con la union de clientes y su geolocalización
df_customers_geo = pd.merge(df_customers,
                            df_unique_geolocations,
                            left_on='customer_zip_code_prefix',
                            right_on='geolocation_zip_code_prefix',
                            how='left')
df_customers_geo.head()

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP,14409,-20.509897,-47.397866,franca,SP
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP,NaN,NaN,NaN,NaN,NaN
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP,NaN,NaN,NaN,NaN,NaN
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP,NaN,NaN,NaN,NaN,NaN
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP,13056,-22.987222,-47.151073,campinas,SP


Podemos notar que en la unión generamos la NaN para diferentes columnas unidas , y esto es debido a que en df_customers no especificamos que customer_zip_code_prefix debe ser de 5 dígitos dado que es un CP , por lo que hay que rellenar con cero a la izquierda aquellos que tengan una longitud menor a 5.

In [ ]:
# del df df_customers definir que la longitud de la columna customer_zip_code_prefix es de 5 dígitos
# y que aquellos que sean menores a 5 rellenar con cero a la izquierda
df_customers['customer_zip_code_prefix'] = df_customers['customer_zip_code_prefix'].str.zfill(5)
df_customers.sample(10)

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
97973,72740dc2b626c46da39cd84cad2102bf,00a1d0d7d91c17c634d6896ce10ce1c6,60711,fortaleza,CE
26012,0841d3b6ce285b44d072153fad3a0e8a,721583925e5e3b7f7a7f201aeeac0bb3,78068,cuiaba,MT
87598,04c79718e69bce669782d1cdbfa6f138,d39668002ff4ce93810dc43d7e765c1d,61760,eusebio,CE
14876,3380dbe062b329f11a5e5de5552c5c34,738ffcf1017b584e9d2684b36e07469c,74440,goiania,GO
49110,6d822462f31cc86382bb8da8b2676f5c,0f0e48a91c9c285404572b095bcb373c,49048,aracaju,SE
18923,29c13f323716a5bbd7356343e45d9868,35e146c985a9cf989c91edb4c0c156fa,21371,rio de janeiro,RJ
68896,b123bc64605b64205b48d31cb1d966c1,e658ba73bfb0a4c88545c2e1ac3db134,18150,ibiuna,SP
93191,ac037fe63acefadb797790ae6d6a217e,413fb06a4ed4b4ae5523b39263ec3063,31810,belo horizonte,MG
28779,75ea3b206fb4890d3247a0afb3cd9f37,f3ec7fe1b723815ea67103324912e409,12922,braganca paulista,SP
89070,e7939cb82c253885e5a3e92585980ac1,b770cc23ba32f2d02183d5fc502e39b8,13215,jundiai,SP


In [ ]:
#Volvemos a crear el dataframe con la union de clientes y su geolocalización
df_customers_geo = pd.merge(df_customers,
                            df_unique_geolocations,
                            left_on='customer_zip_code_prefix',
                            right_on='geolocation_zip_code_prefix',
                            how='left')
df_customers_geo.head()

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP,14409,-20.509897,-47.397866,franca,SP
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,09790,sao bernardo do campo,SP,09790,-23.726853,-46.545746,sao bernardo do campo,SP
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,01151,sao paulo,SP,01151,-23.527788,-46.660310,sao paulo,SP
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,08775,mogi das cruzes,SP,08775,-23.496930,-46.185352,mogi das cruzes,SP
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP,13056,-22.987222,-47.151073,campinas,SP


In [ ]:
df_customers_geo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 10 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   customer_id                  99441 non-null  object 
 1   customer_unique_id           99441 non-null  object 
 2   customer_zip_code_prefix     99441 non-null  object 
 3   customer_city                99441 non-null  object 
 4   customer_state               99441 non-null  object 
 5   geolocation_zip_code_prefix  99163 non-null  object 
 6   geolocation_lat              99163 non-null  float64
 7   geolocation_lng              99163 non-null  float64
 8   geolocation_city             99163 non-null  object 
 9   geolocation_state            99163 non-null  object 
dtypes: float64(2), object(8)
memory usage: 7.6+ MB


In [ ]:
df_customers_geo.isna().sum()

,0
customer_id,0
customer_unique_id,0
customer_zip_code_prefix,0
customer_city,0
customer_state,0
geolocation_zip_code_prefix,278
geolocation_lat,278
geolocation_lng,278
geolocation_city,278
geolocation_state,278


Vemos que hay 278 registros con NaN, que se refiere a que hay algunos CP de clientes que no están en df_geolocation_zip_code_prefix

In [ ]:
#Obtenemos los registros NaN del df df_customers_geo
df_customers_geo[df_customers_geo.isna().any(axis=1)]

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
354,ecb1725b26e8b8c458181455dfa434ea,b55a113bb84fc10eaf58c6d09ec69794,72300,brasilia,DF,NaN,NaN,NaN,NaN,NaN
382,bcf86029aeed4ed8bac0e16eb14c22f5,7cd7974c9f79f75b77f323878ef87f43,11547,cubatao,SP,NaN,NaN,NaN,NaN,NaN
877,f4302056f0c58570522590f8181de2c7,67b05b597a66b5c449025000b9430abb,64605,picos,PI,NaN,NaN,NaN,NaN,NaN
1218,03bbe0ce5c28e05f22917607db798818,8f3dca4306d5a89e4ae2c65c110603a2,72465,brasilia,DF,NaN,NaN,NaN,NaN,NaN
1272,ad4950aded55c2ea376be59506456d68,aa2b96dd03307ea6dc4b763c0b5f0b39,07729,caieiras,SP,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
97467,cf818420383856a129134f5f8343f7b8,795c495a65f983b242fb01bd507977c5,72338,brasilia,DF,NaN,NaN,NaN,NaN,NaN
97780,67f3e907dce402e696b15f9308ff22ed,6f232f2f5c7f33b7bd9d794d2afacadd,68629,paragominas,PA,NaN,NaN,NaN,NaN,NaN
98140,f792e419335df11d82c32efcfb09c51b,c04c085b8e7573ba87b9ae1968d0985e,28530,sao sebastiao do paraiba,RJ,NaN,NaN,NaN,NaN,NaN
98878,78a11bb1fa72f556996b9a5b9bcd0629,e7536f62a200b415edd9491ac12a17fa,55863,siriji,PE,NaN,NaN,NaN,NaN,NaN


In [ ]:
#Solo confirmamos que realmente no existan en el df df_unique_geolocations.query
df_unique_geolocations.query('geolocation_zip_code_prefix == "07729"')

,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state


Ahora vamos a unir la información con el nmbre del estado

In [ ]:
df_customers_geo_estado = pd.merge(df_customers_geo,
                            df_states_abreviation,
                            left_on='geolocation_state',
                            right_on='abbreviation',
                            how='left')

In [ ]:
df_customers_geo_estado

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state,abbreviation,state_name
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP,14409,-20.509897,-47.397866,franca,SP,SP,São Paulo
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,09790,sao bernardo do campo,SP,09790,-23.726853,-46.545746,sao bernardo do campo,SP,SP,São Paulo
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,01151,sao paulo,SP,01151,-23.527788,-46.660310,sao paulo,SP,SP,São Paulo
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,08775,mogi das cruzes,SP,08775,-23.496930,-46.185352,mogi das cruzes,SP,SP,São Paulo
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP,13056,-22.987222,-47.151073,campinas,SP,SP,São Paulo
...,...,...,...,...,...,...,...,...,...,...,...,...
99436,17ddf5dd5d51696bb3d7c6291687be6f,1a29b476fee25c95fbafc67c5ac95cf8,03937,sao paulo,SP,03937,-23.587901,-46.501830,são paulo,SP,SP,São Paulo
99437,e7b71a9017aa05c9a7fd292d714858e8,d52a67c98be1cf6a5c84435bd38d095d,06764,taboao da serra,SP,06764,-23.612294,-46.765787,taboao da serra,SP,SP,São Paulo
99438,5e28dfe12db7fb50a4b2f691faecea5e,e9f50caf99f032f0bf3c55141f019d99,60115,fortaleza,CE,60115,-3.744128,-38.510859,fortaleza,CE,CE,Ceará
99439,56b18e2166679b8a959d72dd06da27f9,73c2643a0a458b49f58cea58833b192e,92120,canoas,RS,92120,-29.956391,-51.167614,canoas,RS,RS,Rio Grande do Sul


Lo siguiente es crear un df con las órdenes y total de articulos y  precios

In [ ]:
#Unir df df_orders y items_agg por order_id
df_orders_totals = pd.merge(df_olist_orders,
                            order_items_agg,
                            left_on='order_id',
                            right_on='order_id',
                            how='left')

In [ ]:
df_orders_totals

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,distance_distribution_center,year,month,quarter,year_month,delta_days,delay_status,total_products,total_sales
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18,29.84,2017,10,2017Q4,2017-10,-7.107488,on_time,1.0,29.99
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13,25.53,2018,7,2018Q3,2018-07,-5.355729,on_time,1.0,118.70
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04,61.56,2018,8,2018Q3,2018-08,-17.245498,on_time,1.0,159.90
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15,60.30,2017,11,2017Q4,2017-11,-12.980069,on_time,1.0,45.00
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26,80.97,2018,2,2018Q1,2018-02,-9.238171,on_time,1.0,19.90
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99436,9c5dedf39a927c1b2549525ed64a053c,39bd1228ee8140590ac3aca26f2dfe00,delivered,2017-03-09 09:54:05,2017-03-09 09:54:05,2017-03-10 11:18:03,2017-03-17 15:08:01,2017-03-28,68.26,2017,3,2017Q1,2017-03,-10.369433,on_time,1.0,72.00
99437,63943bddc261676b46f01ca7ac2f7bd8,1fca14ff2861355f6e5f14306ff977a7,delivered,2018-02-06 12:58:58,2018-02-06 13:10:37,2018-02-07 23:22:42,2018-02-28 17:37:56,2018-03-02,14.92,2018,2,2018Q1,2018-02,-1.265324,on_time,1.0,174.90
99438,83c1379a015df1e13d02aae0204711ab,1aa71eb042121263aafbe80c1b562c9c,delivered,2017-08-27 14:46:43,2017-08-27 15:04:16,2017-08-28 20:52:26,2017-09-21 11:24:17,2017-09-27,36.37,2017,8,2017Q3,2017-08,-5.524803,on_time,1.0,205.99
99439,11c177c8e97725db2631073c19f07b62,b331b74b18dc79bcdf6532d51e1637c1,delivered,2018-01-08 21:28:27,2018-01-08 21:36:21,2018-01-12 15:35:03,2018-01-25 23:32:54,2018-02-15,32.04,2018,1,2018Q1,2018-01,-20.018819,on_time,2.0,359.98


Con estos dos df podemos tenemos las columnas solicitadas en el requerimiento por lo que podemos empezar a unirlos

In [ ]:
# unimos los df df_orders_totals con df_customers_geo_estado por customer_id
df_results = df_orders_totals.merge(
    df_customers_geo_estado,
    on=['customer_id'],
    how='left'
    )

In [ ]:
df_results

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,distance_distribution_center,year,...,customer_zip_code_prefix,customer_city,customer_state,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state,abbreviation,state_name
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18,29.84,2017,...,03149,sao paulo,SP,03149,-23.574809,-46.587471,sao paulo,SP,SP,São Paulo
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13,25.53,2018,...,47813,barreiras,BA,47813,-12.169860,-44.988369,barreiras,BA,BA,Bahia
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04,61.56,2018,...,75265,vianopolis,GO,75265,-16.746337,-48.514624,vianopolis,GO,GO,Goiás
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15,60.30,2017,...,59296,sao goncalo do amarante,RN,59296,-5.767733,-35.275467,sao goncalo do amarante,RN,RN,Rio Grande do Norte
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26,80.97,2018,...,09195,santo andre,SP,09195,-23.675037,-46.524784,santo andre,SP,SP,São Paulo
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99436,9c5dedf39a927c1b2549525ed64a053c,39bd1228ee8140590ac3aca26f2dfe00,delivered,2017-03-09 09:54:05,2017-03-09 09:54:05,2017-03-10 11:18:03,2017-03-17 15:08:01,2017-03-28,68.26,2017,...,12209,sao jose dos campos,SP,12209,-23.178732,-45.889711,são josé dos campos,SP,SP,São Paulo
99437,63943bddc261676b46f01ca7ac2f7bd8,1fca14ff2861355f6e5f14306ff977a7,delivered,2018-02-06 12:58:58,2018-02-06 13:10:37,2018-02-07 23:22:42,2018-02-28 17:37:56,2018-03-02,14.92,2018,...,11722,praia grande,SP,11722,-24.001467,-46.446355,praia grande,SP,SP,São Paulo
99438,83c1379a015df1e13d02aae0204711ab,1aa71eb042121263aafbe80c1b562c9c,delivered,2017-08-27 14:46:43,2017-08-27 15:04:16,2017-08-28 20:52:26,2017-09-21 11:24:17,2017-09-27,36.37,2017,...,45920,nova vicosa,BA,45920,-17.891522,-39.370942,nova vicosa,BA,BA,Bahia
99439,11c177c8e97725db2631073c19f07b62,b331b74b18dc79bcdf6532d51e1637c1,delivered,2018-01-08 21:28:27,2018-01-08 21:36:21,2018-01-12 15:35:03,2018-01-25 23:32:54,2018-02-15,32.04,2018,...,28685,japuiba,RJ,28685,-22.555985,-42.690761,japuiba,RJ,RJ,Rio de Janeiro


In [ ]:
#Observamos las columnas
df_results.columns

Index(['order_id', 'customer_id', 'order_status', 'order_purchase_timestamp',
       'order_approved_at', 'order_delivered_carrier_date',
       'order_delivered_customer_date', 'order_estimated_delivery_date',
       'distance_distribution_center', 'year', 'month', 'quarter',
       'year_month', 'delta_days', 'delay_status', 'total_products',
       'total_sales', 'customer_unique_id', 'customer_zip_code_prefix',
       'customer_city', 'customer_state', 'geolocation_zip_code_prefix',
       'geolocation_lat', 'geolocation_lng', 'geolocation_city',
       'geolocation_state', 'abbreviation', 'state_name'],
      dtype='object')

# 4.- Exportamos el df al archivo de salida: **"oilst_processed.csv"**

In [ ]:
#Creamos el archivo
df_results.to_csv(os.path.join(DATA_PATH, 'oilst_processed.csv'), index=False)

In [ ]:
#Validamos que se haya generado
print(f"Ruta del archivo: {os.path.join(DATA_PATH, 'oilst_processed.csv')}")
os.path.exists(os.path.join(DATA_PATH, 'oilst_processed.csv'))

Ruta del archivo: /content/oilst_processed.csv


True